# 1. Candidate generation

In [437]:
import pandas as pd 
import numpy as np
import random


***Read Files***

In [438]:
df = pd.read_csv('data/user_videos_updated.csv')
# todo: config lại để chỉ drop na tại những cột mình xài
df = df.dropna()
df.count()


user_id               1476
user_gender           1476
user_zip              1476
dwell_time            1476
watchedAt             1476
                      ... 
discussion_density    1476
like_percentage       1476
dislike_percentage    1476
like_dislike_ratio    1476
share_percentage      1476
Length: 120, dtype: int64

In [439]:
# map user id with index
userMap = {v:k for k,v in enumerate(df['user_id'].unique())}
df['user_id'] = df['user_id'].map(userMap)
# map video id with index
videoMap = {v:k for k,v in enumerate(df['video_id'].unique())}
df['video_id'] = df['video_id'].map(videoMap)

In [440]:
# calculate example age
df['example_age']=(pd.to_datetime("now")-pd.to_datetime(df['publishedAt']).dt.tz_localize(None))/ np.timedelta64(1,'D')

In [441]:
def normalize_col(df,col_name):
    df[col_name] = (df[col_name] - df[col_name].mean()) / (df[col_name].std() + 1e-10)
    return df

In [442]:
df['like/dislike'] = [True if x >= 160 else False for _,x in enumerate(df['dwell_time'].tolist())]
df = normalize_col(df,'example_age')
df = normalize_col(df,'dwell_time')
df = normalize_col(df,'watchTime')
test= df.groupby(['user_id'])['video_id'].count()
test

user_id
0       2
1       3
2       1
3       3
4       1
       ..
143     7
144     1
145    10
146     8
147     1
Name: video_id, Length: 148, dtype: int64

In [443]:
df['like/dislike'].value_counts()

False    773
True     703
Name: like/dislike, dtype: int64

In [444]:
users = df.iloc[:,:3]
videos = df.iloc[:,7:]



In [445]:
videos['year'] = videos['publishedAt'].apply(lambda x:str(x).split('-')[0])


In [446]:
watch_cols = ['user_id', 'video_id','dwell_time','like/dislike','watchedAt']
watch = df[watch_cols]
watch['year'] = watch['watchedAt'].apply(lambda x:str(x).split('-')[0])
watch = normalize_col(watch,'dwell_time')

C:\Users\I7\AppData\Local\Temp/ipykernel_14864/4189475514.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  watch['year'] = watch['watchedAt'].apply(lambda x:str(x).split('-')[0])
C:\Users\I7\AppData\Local\Temp/ipykernel_14864/3869978841.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col_name] = (df[col_name] - df[col_name].mean()) / (df[col_name].std() + 1e-10)


In [447]:
# like & dislike both false -> like
# like & dislike both true -> random false
watch['like/dislike'] = df['like/dislike']
watch['like/dislike'].unique()

C:\Users\I7\AppData\Local\Temp/ipykernel_14864/324578433.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  watch['like/dislike'] = df['like/dislike']


array([False,  True])

In [448]:
# genres encoding
genres = df['category_term'].unique().tolist()
genres_encoded = {x: i for i,x in enumerate(genres)}
genres_encoded_2_genres = {i:x for i,x in enumerate(genres)}


# user encoding
user_ids = users['user_id'].unique().tolist()
user2user_encoded = {x: i for i, x in enumerate(user_ids)}
userencoded2user = {i: x for i, x in enumerate(user_ids)}

# movie index encoding
video_ids = videos["video_id"].unique().tolist()
video2video_encoded = {x: i for i, x in enumerate(video_ids)}
video_encoded2video = {i: x for i, x in enumerate(video_ids)}

# movie title encoding
title_ids = videos['video_title'].unique().tolist()
#user_video_fond = like_dislike.pivot(index='user', columns='like/dislike', values='video').reset_index()
#user_video_fond.loc[user_video_fond['like'].isnull(),['like']] = user_video_fond.loc[user_video_fond['like'].isnull(),'like'].apply(lambda x: [dfenc['video'].max()+1] ) = {x: i for i, x in enumerate(title_ids)}
title_encoded2title = {i: x for i, x in enumerate(title_ids)}

#video category encoding 
cats = videos['category_term'].unique().tolist()
cat2cat_encoded = {x: i for i,x in enumerate(cats)}
cat_encoded2cat = {x:i for x,i in enumerate(cats)}

# genders encoding
gender_ids = users['user_gender'].unique().tolist()
gender2gender_encoded = {x: i for i, x in enumerate(gender_ids)}
gender_encoded2gender = {i: x for i, x in enumerate(gender_ids)}

#like/disklike encoding
like_dislike_ids =  watch['like/dislike'].unique().tolist()
like_dislike_ids2encoded = {x: i for i, x in enumerate(like_dislike_ids)}
encoded2like = {i: x for i, x in enumerate(like_dislike_ids)}




In [449]:
# # Only do this one time
likeMap = {True:'like', False:'dislike'}
df['like/dislike'] = df['like/dislike'].map(likeMap)

In [450]:
df['like/dislike']

0       dislike
1          like
2          like
3       dislike
4          like
         ...   
1807       like
1808    dislike
1810       like
1811       like
1813    dislike
Name: like/dislike, Length: 1476, dtype: object

In [451]:

dfenc = df
# Change to encoding
title2title_encoded = {x: i for i, x in enumerate(title_ids)}
dfenc["title_id"] = df["video_title"].map(title2title_encoded)

dfenc["user"] = df["user_id"].map(user2user_encoded)
dfenc["video"] = df["video_id"].map(video2video_encoded)
dfenc["title_d"] = df["video_title"].map(title2title_encoded)
dfenc['gender_d'] = df['user_gender'].map(gender2gender_encoded)
dfenc['cat_d'] = df['category_term'].map(cat2cat_encoded)

watch_hist = dfenc.groupby(['user'])['video_id'].apply(list).reset_index()

watch_hist_time = dfenc.groupby(['user'])['dwell_time'].apply(list).reset_index()

like_dislike = dfenc.groupby(['user','like/dislike'])['video'].apply(list).reset_index()

example_age = dfenc.groupby(['user'])['example_age'].apply(list).reset_index()


user_video_fond = like_dislike.pivot(index='user', columns='like/dislike', values='video').reset_index()
user_video_fond.fillna(dfenc["video"].max()+1, inplace=True)
user_video_fond['dislike'] = user_video_fond['dislike'].apply(lambda x: (x)) 
user_video_fond['like'] = user_video_fond['like'].apply(lambda x: (x)) 
# coi lại chỗ này -> 1435 (max) đang nhiều
user_video_list = dfenc.pivot(index='user_id', columns='video_id', values='video').reset_index()
user_video_fond = like_dislike.pivot(index='user', columns='like/dislike', values='video').reset_index()
user_video_fond.loc[user_video_fond['like'].isnull(),['like']] = user_video_fond.loc[user_video_fond['like'].isnull(),'like'].apply(lambda x: [dfenc['video'].max()+1] )

sample_data=dfenc[['user','user_gender']]
# sample_data=sample_data.reset_index()
sample_data = sample_data.drop_duplicates()

user_movie_list = pd.merge(sample_data,watch_hist, how= 'left')
user_movie_list = pd.merge(user_movie_list,watch_hist_time, how='left')
user_movie_list = pd.merge(user_movie_list,example_age, how='left')
user_movie_list = pd.merge(user_movie_list,user_video_fond,how='left')

# generate predict labels
user_movie_list['predict_labels'] = user_movie_list['video_id'].apply(lambda x: int(random.uniform(0,dfenc["video"].max()))) #label을 마지막 값으로..


# split 60:40
# train_size = user_movie_list.user.max() * 0.6
# # train_size = 10 * 0.6
# test_size = user_movie_list.user.max() - train_size
# train_data = user_movie_list[(user_movie_list.user >= 1)&
#                                   (user_movie_list.user <= tr)]
# test_data = user_movie_list[(user_movie_list.user >= train_size+1)&
#                                   (user_movie_list.user <= 10)]

In [452]:
# user_movie_list['predict_labels'] = user_movie_list['like'].apply(lambda x: (x[-1]) )
# user_movie_list['like'] = user_movie_list['like'].apply(lambda x: x[:-1])

In [453]:
# split test and train set
from sklearn.model_selection import train_test_split

train_data, test_data = train_test_split(user_movie_list, test_size=0.2)


In [454]:
train_data

,user,user_gender,video_id,dwell_time,example_age,dislike,like,predict_labels
134,134,Female,"[1196, 1197, 1198, 1199, 1200, 1201, 1202, 120...","[-0.5782094033644067, -0.5917619984173018, -0....","[-0.06208166045947228, -0.9034198266440422, 0....","[1196, 1197, 1198, 1199, 1202, 1203, 1204, 120...","[1200, 1201, 1206, 1207, 1217, 1220, 1226, 122...",543
89,89,Male,[713],[-0.07332280834048614],[-0.8854909340262294],NaN,[713],206
27,27,Female,"[188, 189, 190, 191, 192, 193, 194, 195, 196, ...","[0.24767522290450272, 1.7209538503971737, -0.3...","[2.907296769590773, -0.145311800946087, 1.2334...","[190, 192, 194, 195, 196, 197, 198, 200, 205, ...","[188, 189, 191, 193, 199, 201, 202, 203, 204, ...",699
143,143,Female,"[1411, 1412, 1413, 1414, 1415, 1416, 1417]","[0.7568929343111044, -0.6447110359630486, 0.17...","[-6.120972349966186e-05, 1.7882759279639249, -...",[1412],"[1411, 1413, 1414, 1415, 1416, 1417]",913
133,133,Female,"[1193, 1194, 995, 1195]","[0.06306632608508295, 0.2563070618706883, 0.05...","[-0.23311489524792484, -0.11528582007263405, -...",NaN,"[1193, 1194, 995, 1195]",1146
...,...,...,...,...,...,...,...,...
142,142,Female,"[1390, 1391, 1392, 1393, 1394, 1395, 1396, 139...","[-0.08698178169049968, -0.07330153268106251, 0...","[1.7227039115663512, 0.7423296934958185, 0.517...","[1393, 1395, 1396, 1397, 1401, 1404, 1405, 140...","[1390, 1391, 1392, 1394, 1398, 1399, 1400, 140...",735
52,52,Female,[375],[-0.5285459355096623],[0.2526833093909996],[375],[1435],209
32,32,Female,"[221, 222, 223, 224, 225, 226, 227, 228, 229, ...","[-0.45135176436065394, -0.4621689174876451, 0....","[-0.4381018074354285, -0.746538629400452, 1.17...","[221, 222, 226, 227, 228, 230, 232, 234, 239, ...","[223, 224, 225, 229, 231, 233, 235, 236, 237, ...",299
53,53,Female,"[376, 377, 378]","[-0.30483237666948737, -0.6750258112618993, -0...","[-0.5874282592941824, -0.0378823434514022, -0....","[376, 377, 378]",[1435],27


In [455]:
user_movie_list

,user,user_gender,video_id,dwell_time,example_age,dislike,like,predict_labels
0,0,Female,"[0, 1]","[-0.3868318074683337, 1.652215234179126]","[-0.5473293143225252, 0.3060151782212582]",[0],[1],1423
1,1,Male,"[2, 3, 4]","[0.11961398945335214, -0.6393313335087709, -0....","[-0.7498156961033203, -0.805355604642793, 0.08...",[3],"[2, 4]",441
2,2,Male,[5],[-0.12214436795803571],[0.8014712518426899],NaN,[5],1193
3,3,Female,"[6, 7, 8]","[-0.6794602665617813, 0.2514288571028265, 0.22...","[2.2943273329718292, 1.6946288563179348, 1.695...",[6],"[7, 8]",730
4,4,Female,[9],[-0.5346581285240988],[2.0662103271083283],[9],[1435],785
...,...,...,...,...,...,...,...,...
143,143,Female,"[1411, 1412, 1413, 1414, 1415, 1416, 1417]","[0.7568929343111044, -0.6447110359630486, 0.17...","[-6.120972349966186e-05, 1.7882759279639249, -...",[1412],"[1411, 1413, 1414, 1415, 1416, 1417]",913
144,144,Female,[219],[-0.16083871368990538],[-0.7801721145359796],NaN,[219],916
145,145,Female,"[1418, 1419, 1420, 1421, 214, 1422, 39, 1423, ...","[-0.009772413641902929, -0.6806669003890967, 0...","[-0.8572521146874063, -0.9136670678625444, 1.7...","[1419, 1421, 214, 1422, 1423, 1425]","[1418, 1420, 39, 1424]",744
146,146,Female,"[1426, 1427, 1428, 1429, 1430, 1431, 1432, 1433]","[0.12305152814023815, -0.4512453860635355, 0.0...","[-0.37610211502503044, -0.3609177122240282, -0...","[1427, 1431]","[1426, 1428, 1429, 1430, 1432, 1433]",348


### Hyperparameter Definition

In [456]:
# CONTINUE FROM HERE - 30-09-2021
EMBEDDING_DIMS = 16
DENSE_UNITS = 64
DROPOUT_PCT = 0.0
ALPHA = 0.0
NUM_CLASSES= df["video_id"].max() + 2
LEARNING_RATE = 0.001

In [457]:

import tensorflow as tf
tf.random.set_seed(
    1024
)
class MaskedEmbeddingsAggregatorLayer(tf.keras.layers.Layer):
    def __init__(self, agg_mode='sum', **kwargs):
        super(MaskedEmbeddingsAggregatorLayer, self).__init__(**kwargs)

        if agg_mode not in ['sum', 'mean']:
            raise NotImplementedError('mode {} not implemented!'.format(agg_mode))
        self.agg_mode = agg_mode
    
    @tf.function
    def call(self, inputs, mask=None):
#         https://www.tensorflow.org/api_docs/python/tf/ragged/boolean_mask
        masked_embeddings = tf.ragged.boolean_mask(inputs, mask)
        if self.agg_mode == 'sum':
            aggregated =  tf.reduce_sum(masked_embeddings, axis=1)
        elif self.agg_mode == 'mean':
            aggregated = tf.reduce_mean(masked_embeddings, axis=1)
        return aggregated
    
    def get_config(self):
        # this is used when loading a saved model that uses a custom layer
        return {'agg_mode': self.agg_mode}
    
class L2NormLayer(tf.keras.layers.Layer):
    def __init__(self, **kwargs):
        super(L2NormLayer, self).__init__(**kwargs)
    
    @tf.function
    def call(self, inputs, mask=None):
        if mask is not None:
            inputs = tf.ragged.boolean_mask(inputs, mask).to_tensor()
        return tf.math.l2_normalize(inputs, axis=-1)

    def compute_mask(self, inputs, mask):
        return mask

In [458]:
#---inputs
import tensorflow as tf
import datetime
import os
input_watch_hist = tf.keras.Input(shape=(None, ), name='watch_hist')
input_dwell_time = tf.keras.layers.Input(shape=(None,), name='dwell_time')
input_like = tf.keras.layers.Input(shape=(None,), name='like')
input_example_age = tf.keras.Input(shape=(None, ), name='example_age')
# input_occupation = tf.keras.Input(shape=(None, ), name='occupation')


#--- layers
features_embedding_layer = tf.keras.layers.Embedding(input_dim=NUM_CLASSES, output_dim=EMBEDDING_DIMS, 
                                            mask_zero=True, trainable=True, name='features_embeddings')
labels_embedding_layer = tf.keras.layers.Embedding(input_dim=NUM_CLASSES, output_dim=EMBEDDING_DIMS, 
                                            mask_zero=True, trainable=True, name='labels_embeddings')

avg_embeddings = MaskedEmbeddingsAggregatorLayer(agg_mode='mean', name='aggregate_embeddings')

dense_1 = tf.keras.layers.Dense(units=DENSE_UNITS, name='dense_1')
dense_2 = tf.keras.layers.Dense(units=DENSE_UNITS, name='dense_2')
dense_3 = tf.keras.layers.Dense(units=DENSE_UNITS, name='dense_3')
l2_norm_1 = L2NormLayer(name='l2_norm_1')

dense_output = tf.keras.layers.Dense(NUM_CLASSES, activation=tf.nn.softmax, name='dense_output')

#--- features
features_embeddings = features_embedding_layer(input_watch_hist)
l2_norm_features = l2_norm_1(features_embeddings)
avg_features = avg_embeddings(l2_norm_features)

labels_watch_embeddings = labels_embedding_layer(input_dwell_time)
l2_norm_watched = l2_norm_1(labels_watch_embeddings)
avg_watched = avg_embeddings(l2_norm_watched)

labels_like_embeddings = labels_embedding_layer(input_like)
l2_norm_like = l2_norm_1(labels_like_embeddings)
avg_liked = avg_embeddings(l2_norm_like)

labels_example_age_embeddings = labels_embedding_layer(input_example_age)
l2_norm_example_age = l2_norm_1(labels_example_age_embeddings)
avg_example_age = avg_embeddings(l2_norm_example_age)

# labels_occupation_embeddings = labels_embedding_layer(input_occupation)
# l2_norm_occupation = l2_norm_1(labels_occupation_embeddings)
# avg__occupation = avg_embeddings(l2_norm_occupation)


print(avg_features)
print(avg_watched)
print(avg_liked)
print(avg_example_age)
# print(input_occupation)

# 임베딩 벡터들 연결
concat_inputs = tf.keras.layers.Concatenate(axis=1)([avg_features,
                                                     avg_watched,
                                                     avg_liked,
                                                     avg_example_age,
#                                                      avg__occupation
                                                     ])
# Dense Layers
dense_1_features = dense_1(concat_inputs)
dense_1_relu = tf.keras.layers.ReLU(name='dense_1_relu')(dense_1_features)
dense_1_batch_norm = tf.keras.layers.BatchNormalization(name='dense_1_batch_norm')(dense_1_relu)

dense_2_features = dense_2(dense_1_relu)
dense_2_relu = tf.keras.layers.ReLU(name='dense_2_relu')(dense_2_features)
# dense_2_batch_norm = tf.keras.layers.BatchNormalization(name='dense_2_batch_norm')(dense_2_relu)

dense_3_features = dense_3(dense_2_relu)
dense_3_relu = tf.keras.layers.ReLU(name='dense_3_relu')(dense_3_features)
dense_3_batch_norm = tf.keras.layers.BatchNormalization(name='dense_3_batch_norm')(dense_3_relu)
# for ranking we use this line
# outputs = dense_output(dense_3_batch_norm) + tf.keras.layers.Softmax(dense_3_batch_norm)
# for canndidate generation
outputs = dense_output(dense_3_batch_norm)


#Optimizer
optimiser = tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE)

#--- prep model
model = tf.keras.models.Model(
    inputs=[input_watch_hist, 
            input_dwell_time, 
            input_like,
            input_example_age,
#             input_occupation,
            ],
    outputs=[outputs]
)
logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)
et = tf.keras.callbacks.EarlyStopping(
    monitor="acc",
    #min_delta=0,
    patience=50,
    verbose=0,
    mode="auto",
    baseline=None,
    restore_best_weights=False,
)

# đổi tử sparse -> categorical:
# đổi predict labels từ scalar (1 int) -> one hot encoding.
# vd [1],[2],[3] -> [100],[010], [001]

# model.compile(optimizer=optimiser, loss = lambda y_pred,y_true: tf.keras.losses.CategoricalCrossentropy()(tf.keras.layers.Softmax(y_pred), y) )
model.compile(optimizer=optimiser, loss='sparse_categorical_crossentropy', metrics=['acc'])

model.summary()

KerasTensor(type_spec=TensorSpec(shape=(None, 16), dtype=tf.float32, name=None), name='aggregate_embeddings/PartitionedCall:0', description="created by layer 'aggregate_embeddings'")
KerasTensor(type_spec=TensorSpec(shape=(None, 16), dtype=tf.float32, name=None), name='aggregate_embeddings/PartitionedCall:0', description="created by layer 'aggregate_embeddings'")
KerasTensor(type_spec=TensorSpec(shape=(None, 16), dtype=tf.float32, name=None), name='aggregate_embeddings/PartitionedCall:0', description="created by layer 'aggregate_embeddings'")
KerasTensor(type_spec=TensorSpec(shape=(None, 16), dtype=tf.float32, name=None), name='aggregate_embeddings/PartitionedCall:0', description="created by layer 'aggregate_embeddings'")
Model: "model_15"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
watch_hist (InputLayer)         [(None, None)]      

In [459]:
tf.keras.utils.plot_model(model, show_shapes=True, show_layer_names=True,dpi=96)

('You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) ', 'for plot_model/model_to_dot to work.')


In [460]:
train_data

,user,user_gender,video_id,dwell_time,example_age,dislike,like,predict_labels
134,134,Female,"[1196, 1197, 1198, 1199, 1200, 1201, 1202, 120...","[-0.5782094033644067, -0.5917619984173018, -0....","[-0.06208166045947228, -0.9034198266440422, 0....","[1196, 1197, 1198, 1199, 1202, 1203, 1204, 120...","[1200, 1201, 1206, 1207, 1217, 1220, 1226, 122...",543
89,89,Male,[713],[-0.07332280834048614],[-0.8854909340262294],NaN,[713],206
27,27,Female,"[188, 189, 190, 191, 192, 193, 194, 195, 196, ...","[0.24767522290450272, 1.7209538503971737, -0.3...","[2.907296769590773, -0.145311800946087, 1.2334...","[190, 192, 194, 195, 196, 197, 198, 200, 205, ...","[188, 189, 191, 193, 199, 201, 202, 203, 204, ...",699
143,143,Female,"[1411, 1412, 1413, 1414, 1415, 1416, 1417]","[0.7568929343111044, -0.6447110359630486, 0.17...","[-6.120972349966186e-05, 1.7882759279639249, -...",[1412],"[1411, 1413, 1414, 1415, 1416, 1417]",913
133,133,Female,"[1193, 1194, 995, 1195]","[0.06306632608508295, 0.2563070618706883, 0.05...","[-0.23311489524792484, -0.11528582007263405, -...",NaN,"[1193, 1194, 995, 1195]",1146
...,...,...,...,...,...,...,...,...
142,142,Female,"[1390, 1391, 1392, 1393, 1394, 1395, 1396, 139...","[-0.08698178169049968, -0.07330153268106251, 0...","[1.7227039115663512, 0.7423296934958185, 0.517...","[1393, 1395, 1396, 1397, 1401, 1404, 1405, 140...","[1390, 1391, 1392, 1394, 1398, 1399, 1400, 140...",735
52,52,Female,[375],[-0.5285459355096623],[0.2526833093909996],[375],[1435],209
32,32,Female,"[221, 222, 223, 224, 225, 226, 227, 228, 229, ...","[-0.45135176436065394, -0.4621689174876451, 0....","[-0.4381018074354285, -0.746538629400452, 1.17...","[221, 222, 226, 227, 228, 230, 232, 234, 239, ...","[223, 224, 225, 229, 231, 233, 235, 236, 237, ...",299
53,53,Female,"[376, 377, 378]","[-0.30483237666948737, -0.6750258112618993, -0...","[-0.5874282592941824, -0.0378823434514022, -0....","[376, 377, 378]",[1435],27


In [461]:
history = model.fit([tf.keras.preprocessing.sequence.pad_sequences(train_data['video_id']),
           tf.keras.preprocessing.sequence.pad_sequences(train_data['dwell_time'], dtype=float),
           tf.keras.preprocessing.sequence.pad_sequences(train_data['like']) + 1e-10,
           tf.keras.preprocessing.sequence.pad_sequences(train_data['example_age'], dtype=float),
#            tf.keras.preprocessing.sequence.pad_sequences(train_data['occupation'], dtype=float),
           ],train_data['predict_labels'].values,
           steps_per_epoch=1, epochs=5000,callbacks=[et])




Epoch 1/5000
1/1 [==============================] - 1s 1s/step - loss: 7.2660 - acc: 0.0000e+00
Epoch 2/5000
1/1 [==============================] - 0s 20ms/step - loss: 7.2342 - acc: 0.0000e+00
Epoch 3/5000
1/1 [==============================] - 0s 16ms/step - loss: 7.2047 - acc: 0.0169
Epoch 4/5000
1/1 [==============================] - 0s 13ms/step - loss: 7.1772 - acc: 0.0339
Epoch 5/5000
1/1 [==============================] - 0s 16ms/step - loss: 7.1492 - acc: 0.0508
Epoch 6/5000
1/1 [==============================] - 0s 14ms/step - loss: 7.1203 - acc: 0.0593
Epoch 7/5000
1/1 [==============================] - 0s 14ms/step - loss: 7.0894 - acc: 0.0678
Epoch 8/5000
1/1 [==============================] - 0s 14ms/step - loss: 7.0567 - acc: 0.0847
Epoch 9/5000
1/1 [==============================] - 0s 14ms/step - loss: 7.0236 - acc: 0.1017
Epoch 10/5000
1/1 [==============================] - 0s 14ms/step - loss: 6.9885 - acc: 0.1017
Epoch 11/5000
1/1 [==============================] - 

1/1 [==============================] - 0s 13ms/step - loss: 3.3253 - acc: 0.3983
Epoch 88/5000
1/1 [==============================] - 0s 11ms/step - loss: 3.2914 - acc: 0.3983
Epoch 89/5000
1/1 [==============================] - 0s 11ms/step - loss: 3.2587 - acc: 0.4068
Epoch 90/5000
1/1 [==============================] - 0s 11ms/step - loss: 3.2261 - acc: 0.4153
Epoch 91/5000
1/1 [==============================] - 0s 13ms/step - loss: 3.1944 - acc: 0.4407
Epoch 92/5000
1/1 [==============================] - 0s 13ms/step - loss: 3.1625 - acc: 0.4407
Epoch 93/5000
1/1 [==============================] - 0s 15ms/step - loss: 3.1323 - acc: 0.4322
Epoch 94/5000
1/1 [==============================] - 0s 15ms/step - loss: 3.1013 - acc: 0.4492
Epoch 95/5000
1/1 [==============================] - 0s 11ms/step - loss: 3.0715 - acc: 0.4661
Epoch 96/5000
1/1 [==============================] - 0s 11ms/step - loss: 3.0418 - acc: 0.4661
Epoch 97/5000
1/1 [==============================] - 0s 13ms/ste

1/1 [==============================] - 0s 10ms/step - loss: 1.4456 - acc: 0.7203
Epoch 173/5000
1/1 [==============================] - 0s 14ms/step - loss: 1.4325 - acc: 0.7373
Epoch 174/5000
1/1 [==============================] - 0s 14ms/step - loss: 1.4171 - acc: 0.7373
Epoch 175/5000
1/1 [==============================] - 0s 15ms/step - loss: 1.4040 - acc: 0.7288
Epoch 176/5000
1/1 [==============================] - 0s 14ms/step - loss: 1.3899 - acc: 0.7458
Epoch 177/5000
1/1 [==============================] - 0s 14ms/step - loss: 1.3765 - acc: 0.7542
Epoch 178/5000
1/1 [==============================] - 0s 14ms/step - loss: 1.3642 - acc: 0.7627
Epoch 179/5000
1/1 [==============================] - 0s 11ms/step - loss: 1.3504 - acc: 0.7627
Epoch 180/5000
1/1 [==============================] - 0s 12ms/step - loss: 1.3402 - acc: 0.7627
Epoch 181/5000
1/1 [==============================] - 0s 18ms/step - loss: 1.3288 - acc: 0.7797
Epoch 182/5000
1/1 [==============================] - 0

In [462]:
model.save("candidate_generation.h5")

D:\conda\lib\site-packages\keras\utils\generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


In [463]:
pred = model.predict([tf.keras.preprocessing.sequence.pad_sequences(test_data['video_id']),
           tf.keras.preprocessing.sequence.pad_sequences(test_data['dwell_time'], dtype=float),
           tf.keras.preprocessing.sequence.pad_sequences(test_data['like']),
           tf.keras.preprocessing.sequence.pad_sequences(test_data['example_age'], dtype=float)
           ])

In [464]:
pred

array([[1.06051826e-04, 1.01946935e-04, 1.02730402e-04, ...,
        1.09972883e-04, 1.13572110e-04, 9.30706956e-05],
       [1.06051826e-04, 1.01946935e-04, 1.02730402e-04, ...,
        1.09972883e-04, 1.13572110e-04, 9.30706956e-05],
       [1.06051826e-04, 1.01946935e-04, 1.02730402e-04, ...,
        1.09972883e-04, 1.13572110e-04, 9.30706956e-05],
       ...,
       [1.06051826e-04, 1.01946935e-04, 1.02730402e-04, ...,
        1.09972883e-04, 1.13572110e-04, 9.30706956e-05],
       [1.06051826e-04, 1.01946935e-04, 1.02730402e-04, ...,
        1.09972883e-04, 1.13572110e-04, 9.30706956e-05],
       [1.06051826e-04, 1.01946935e-04, 1.02730402e-04, ...,
        1.09972883e-04, 1.13572110e-04, 9.30706956e-05]], dtype=float32)

In [465]:
# candidate generation: 
###### We extract the top-7 recommendation data for each user.

#???????????????????????????????????????????????????????????????????
N = 30
k = np.sort((-pred).argsort()[:,:N])
print(k)
k = k.flatten()
k[k>df["video"].max()]=0
k = np.unique(k)


[[  69   81  132  140  152  190  261  292  303  348  353  368  414  650
   674  699  713  735  744  757  842  913  947  978 1081 1146 1232 1288
  1353 1376]
 [  69   81  132  140  152  190  261  292  303  348  353  368  414  650
   674  699  713  735  744  757  842  913  947  978 1081 1146 1232 1288
  1353 1376]
 [  69   81  132  140  152  190  261  292  303  348  353  368  414  650
   674  699  713  735  744  757  842  913  947  978 1081 1146 1232 1288
  1353 1376]
 [  69   81  132  140  152  190  261  292  303  348  353  368  414  650
   674  699  713  735  744  757  842  913  947  978 1081 1146 1232 1288
  1353 1376]
 [  69   81  132  140  152  190  261  292  303  348  353  368  414  650
   674  699  713  735  744  757  842  913  947  978 1081 1146 1232 1288
  1353 1376]
 [  69   81  132  140  152  190  261  292  303  348  353  414  650  674
   699  713  735  744  757  842  913  947  978 1081 1146 1232 1260 1288
  1350 1376]
 [  69   81  132  140  152  190  261  292  303  348  353  

In [466]:
k

array([  69,   76,   81,  123,  128,  132,  140,  152,  190,  261,  292,
        303,  348,  353,  368,  386,  414,  449,  523,  543,  650,  674,
        699,  713,  718,  735,  744,  757,  842,  847,  855,  905,  913,
        947,  978, 1020, 1034, 1054, 1075, 1081, 1146, 1170, 1232, 1240,
       1260, 1288, 1319, 1325, 1350, 1353, 1376, 1423], dtype=int64)

In [467]:
title_encoded2title[42]
df.query('video_id == 170')

,user_id,user_gender,user_zip,dwell_time,watchedAt,like,dislike,video_id,video_title,viewCount,...,like_dislike_ratio,share_percentage,example_age,like/dislike,title_id,user,video,title_d,gender_d,cat_d
209,24,Male,2134,-0.114981,2014-06-15 19:12:50.653000+00:00,False,False,170,Kelly Clarkson - Stronger (What Doesn't Kill Y...,110865244,...,0.979833,0.004808,0.492514,like,170,24,170,170,1,3


# 2. Ranking

### Load dmodel and preprocess data

In [468]:
model = tf.keras.models.load_model(
    'candidate_generation.h5',
    custom_objects={
        'L2NormLayer': L2NormLayer,
        "MaskedEmbeddingsAggregatorLayer": MaskedEmbeddingsAggregatorLayer
    }
)

# custom_object: not available in tensorflow 

In [469]:
def get_genres(movies, genres):
    def get_all_genres(gs):
        active = [str(genres_encoded[genre]) for genre, g in zip(genres, gs) if g==1]
        if len(active) == 0:
            return '0'
        return ','.join((active))
    movies['category_term	'] = [
    get_all_genres(gs) for gs in zip(*[movies[genre] for genre in genres])]

In [470]:
title2title_encoded

{'Breastfeeding and your period': 0,
 'Lesson 1: Cloth Diaper Styles - Cloth Diapering 101': 1,
 'Kei To The City [Drift Feature Film - Japan]': 2,
 'MCM QLD Meet 2014': 3,
 'Game of Thrones - Lindsey Stirling & Peter Hollens (Cover)': 4,
 "Air France Flight 447's harrowing end": 5,
 'Sarah McLachlan - Possession [Official Music Video]': 6,
 'Jeff Buckley - Last Goodbye (Edit)': 7,
 'Jeff Buckley - So Real': 8,
 'Huey Lewis And The News - I Want A New Drug (Official Music Video)': 9,
 'Phil Wickham - This Is Amazing Grace (Official Music Video)': 10,
 'Are These Random Facts True or False?': 11,
 'Ingrid Michaelson - "Maybe" (Official Music Video)': 12,
 "'Breaking Bad's' Laura Fraser on the Remainder of Season 5": 13,
 'Red Hot Chili Peppers - Under The Bridge [Video]': 14,
 "'Pussy, get out of my pants' comment has tv hosts losing it": 15,
 'Citizenship Test': 16,
 'Borat speaking hebrew with israeli journalist': 17,
 'The Ultimate Fighter 17: Knockout of the Season': 18,
 'GLORY 16 

In [471]:
genres_encoded

{'Education': 0,
 'People & Blogs': 1,
 'Autos & Vehicles': 2,
 'Music': 3,
 'News & Politics': 4,
 'Entertainment': 5,
 'Comedy': 6,
 'Sports': 7,
 'Film & Animation': 8,
 'Pets & Animals': 9,
 'Howto & Style': 10,
 'Science & Technology': 11,
 'Travel & Events': 12,
 'Gaming': 13,
 'Shows': 14,
 'Nonprofits & Activism': 15}

In [472]:
videos


,video_id,video_title,viewCount,favoriteCount,dislikeCount,likeCount,duration,publishedAt,elapsed_time,category_term,...,engagement,general_engagement,discussion_density,like_percentage,dislike_percentage,like_dislike_ratio,share_percentage,example_age,like/dislike,year
0,0,Breastfeeding and your period,354581,0,169,435,294,2013-10-18T14:45:57.000Z,235,Education,...,0.337845,0.194216,0.058824,0.001227,0.000477,0.720199,0.000017,-0.547329,False,2013
1,1,Lesson 1: Cloth Diaper Styles - Cloth Diaperin...,37647,0,12,404,742,2012-04-13T06:25:23.000Z,798,People & Blogs,...,1.000000,0.000000,0.030303,0.010731,0.000319,0.971154,0.000717,0.306015,True,2012
2,2,Kei To The City [Drift Feature Film - Japan],853761,0,571,48772,3756,2014-02-26T22:00:01.000Z,82,Autos & Vehicles,...,0.070988,0.340177,0.046490,0.057126,0.000669,0.988428,0.001894,-0.749816,True,2014
3,3,MCM QLD Meet 2014,232819,0,68,6382,449,2014-04-03T22:21:04.000Z,46,Autos & Vehicles,...,0.037697,0.644937,0.150376,0.027412,0.000292,0.989457,0.000374,-0.805356,False,2014
4,4,Game of Thrones - Lindsey Stirling & Peter Hol...,4621920,0,1551,125076,196,2012-09-05T18:41:34.000Z,621,Music,...,0.882449,0.609950,0.130719,0.027061,0.000336,0.987751,0.004370,0.081615,True,2012
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1807,1430,Dear 40...,111320,0,40,11280,176,2013-12-09T00:56:55.000Z,179,Comedy,...,1.000000,0.736436,0.036154,0.101330,0.000359,0.996466,0.000153,-0.626633,True,2013
1808,1431,"Toro Y Moi ""Talamak""",2035377,0,157,15173,161,2010-03-09T17:30:08.000Z,1550,Music,...,0.508373,0.000000,0.018367,0.007455,0.000077,0.989759,0.000000,1.486589,False,2010
1810,1432,Freegans: Creative Living Outside of Capitalism,65765,0,29,924,535,2012-10-17T16:58:04.000Z,584,Nonprofits & Activism,...,1.000000,0.527875,0.059322,0.014050,0.000441,0.969570,0.004166,0.016956,True,2012
1811,1433,Always That ONE THING...,73632,0,22,5230,203,2014-05-31T19:38:42.000Z,26,People & Blogs,...,1.000000,0.711601,0.061828,0.071029,0.000299,0.995811,0.000054,-0.894626,True,2014


In [473]:
video_data = videos.set_index(['video_id']).sort_index()
video_data = video_data.loc[k+1]
video_data



,video_title,viewCount,favoriteCount,dislikeCount,likeCount,duration,publishedAt,elapsed_time,category_term,watchTime,...,engagement,general_engagement,discussion_density,like_percentage,dislike_percentage,like_dislike_ratio,share_percentage,example_age,like/dislike,year
video_id,,,,,,,,,,,,,,,,,,,,,
70,13 Facts That Will Make Your Day Better,789502,0,828,9376,105,2013-09-04T00:10:40.000Z,278,Entertainment,-0.101988,...,1.000000,0.836253,0.068918,0.011876,0.001049,0.918855,0.000246,-0.478537,False,2013
77,airport express unboxing and setup,224697,0,124,216,498,2010-05-22T17:30:20.000Z,1532,Science & Technology,-0.121710,...,0.032753,0.000000,0.007576,0.000961,0.000552,0.635294,0.000000,1.372470,False,2010
82,Nonna's Pasta with Zucchini & Tuna - Laura & N...,25941,0,76,13313,683,2014-07-03T22:25:54.000Z,1,Howto & Style,-0.118288,...,1.000000,0.677962,0.052326,0.513203,0.002930,0.994324,0.000424,-0.945696,True,2014
124,John Krasinki Tells All to Ellen,530212,0,34,2505,390,2010-10-07T18:16:50.000Z,1363,Entertainment,-0.121710,...,0.506500,0.000000,0.007407,0.004725,0.000064,0.986609,0.000000,1.159604,True,2010
129,J.Lo Shows Her Amazing Dance Skills,530481,0,251,2171,157,2013-11-25T22:10:13.000Z,198,Entertainment,-0.105665,...,1.000000,0.677163,0.291667,0.004093,0.000473,0.896367,0.000251,-0.606407,True,2013
133,Brandi Glanville Says Joanna Krupa's P*ssy Sme...,229498,0,202,1659,279,2013-11-12T22:52:40.000Z,216,Entertainment,-0.109610,...,0.417330,0.664232,0.157576,0.007229,0.000880,0.891456,0.000423,-0.586404,False,2013
141,John Krasinski + Gun Range = Hot First Date | ...,666361,0,67,6567,283,2012-02-03T16:57:05.000Z,880,Entertainment,-0.121710,...,1.000000,0.000000,0.083045,0.009855,0.000101,0.989901,0.000117,0.413289,True,2012
153,Oh' Pit Crew with the RuPaul's Drag Race Scruf...,123040,0,44,1584,256,2014-03-06T20:02:30.000Z,76,Entertainment,-0.116155,...,0.040313,0.619928,0.122137,0.012874,0.000358,0.972973,0.001130,-0.762027,False,2014
191,Dru Hill - In My Bed,2016304,0,531,15964,236,2010-09-29T22:55:24.000Z,1356,Music,-0.121710,...,0.932390,0.000000,0.059921,0.007917,0.000263,0.967808,0.000000,1.171643,True,2010


In [474]:
video_data["genre_d"] = video_data["category_term"].map(genres_encoded)
video_data["title_d"] = video_data["video_title"].map(title2title_encoded)
video_data

,video_title,viewCount,favoriteCount,dislikeCount,likeCount,duration,publishedAt,elapsed_time,category_term,watchTime,...,discussion_density,like_percentage,dislike_percentage,like_dislike_ratio,share_percentage,example_age,like/dislike,year,genre_d,title_d
video_id,,,,,,,,,,,,,,,,,,,,,
70,13 Facts That Will Make Your Day Better,789502,0,828,9376,105,2013-09-04T00:10:40.000Z,278,Entertainment,-0.101988,...,0.068918,0.011876,0.001049,0.918855,0.000246,-0.478537,False,2013,5,70
77,airport express unboxing and setup,224697,0,124,216,498,2010-05-22T17:30:20.000Z,1532,Science & Technology,-0.121710,...,0.007576,0.000961,0.000552,0.635294,0.000000,1.372470,False,2010,11,77
82,Nonna's Pasta with Zucchini & Tuna - Laura & N...,25941,0,76,13313,683,2014-07-03T22:25:54.000Z,1,Howto & Style,-0.118288,...,0.052326,0.513203,0.002930,0.994324,0.000424,-0.945696,True,2014,10,82
124,John Krasinki Tells All to Ellen,530212,0,34,2505,390,2010-10-07T18:16:50.000Z,1363,Entertainment,-0.121710,...,0.007407,0.004725,0.000064,0.986609,0.000000,1.159604,True,2010,5,124
129,J.Lo Shows Her Amazing Dance Skills,530481,0,251,2171,157,2013-11-25T22:10:13.000Z,198,Entertainment,-0.105665,...,0.291667,0.004093,0.000473,0.896367,0.000251,-0.606407,True,2013,5,129
133,Brandi Glanville Says Joanna Krupa's P*ssy Sme...,229498,0,202,1659,279,2013-11-12T22:52:40.000Z,216,Entertainment,-0.109610,...,0.157576,0.007229,0.000880,0.891456,0.000423,-0.586404,False,2013,5,133
141,John Krasinski + Gun Range = Hot First Date | ...,666361,0,67,6567,283,2012-02-03T16:57:05.000Z,880,Entertainment,-0.121710,...,0.083045,0.009855,0.000101,0.989901,0.000117,0.413289,True,2012,5,141
153,Oh' Pit Crew with the RuPaul's Drag Race Scruf...,123040,0,44,1584,256,2014-03-06T20:02:30.000Z,76,Entertainment,-0.116155,...,0.122137,0.012874,0.000358,0.972973,0.001130,-0.762027,False,2014,5,153
191,Dru Hill - In My Bed,2016304,0,531,15964,236,2010-09-29T22:55:24.000Z,1356,Music,-0.121710,...,0.059921,0.007917,0.000263,0.967808,0.000000,1.171643,True,2010,3,191


In [475]:
#video_data['video_id']=video_data['title_d']
new_data = video_data.merge(df[['user_id','video_id']], on='video_id') 

#video_data = video_data.reindex(columns=[ 'video_id', 'user_id','watchTime', 'genre_d', 'title_d','like/dislike'])
new_data= new_data[[ 'video_id', 'user_id','watchTime', 'genre_d', 'title_d','like/dislike']]
new_data['like/dislike'] = new_data['like/dislike'].map(likeMap)
new_data

,video_id,user_id,watchTime,genre_d,title_d,like/dislike
0,70,17,-0.101988,5,70,dislike
1,77,20,-0.121710,11,77,dislike
2,82,20,-0.118288,10,82,like
3,124,22,-0.121710,5,124,like
4,129,22,-0.105665,5,129,like
5,133,22,-0.109610,5,133,dislike
6,141,22,-0.121710,5,141,like
7,153,24,-0.116155,5,153,dislike
8,191,27,-0.121710,3,191,like
9,262,33,-0.039813,10,262,like


In [476]:
video_list = new_data.groupby(['user_id','like/dislike'])['video_id'].apply(list).reset_index()

video_list

,user_id,like/dislike,video_id
0,17,dislike,[70]
1,20,dislike,[77]
2,20,like,[82]
3,22,dislike,[133]
4,22,like,"[124, 129, 141]"
5,24,dislike,[153]
6,27,like,[191]
7,33,like,[262]
8,41,like,[293]
9,42,dislike,[304]


In [477]:

genre_list = new_data.groupby(['user_id'])['genre_d'].unique().apply(list).reset_index()
#genre_list['genre_d']=genre_list['genre_d'].apply(lambda x: list(set(','.join(x))) ) # 중복제거
#genre_list['genre_d']=genre_list['genre_d'].apply(lambda x:[ x for x in x if x.isdigit() ])

new_data = normalize_col(new_data, 'watchTime')
timestamp_list = new_data.groupby(['user_id'])['watchTime'].unique().apply(list).reset_index()

title_list = new_data.groupby(['user_id'])['title_d'].apply(list).reset_index()
print(video_list)
dataset = video_list.pivot(index='user_id', columns='like/dislike', values='video_id').reset_index()
dataset.fillna(new_data["video_id"].max()+1, inplace=True)

dataset['like'] =dataset['like'].apply(lambda x: x if type(x) is list else [])
dataset['dislike'] =dataset['dislike'].apply(lambda x: x if type(x) is list else [])

dataset = pd.merge(dataset, title_list, how='left')
dataset = pd.merge(dataset, genre_list, how='left')
dataset = pd.merge(dataset, timestamp_list, how='left')

dataset['predict_labels'] = dataset['like'].apply(lambda x: int(random.uniform(1,new_data["video_id"].max()))) #label을 마지막 값으로..

dataset['like']=dataset['like'].apply(lambda x: [new_data["video_id"].max()+1] if x == [] else x)
dataset['dislike']=dataset['dislike'].apply(lambda x: [new_data["video_id"].max()+1] if x == [] else x)
#train_data=dataset[(dataset.user_id >= 1)& (dataset.user_id <= 5)]
#test_data=dataset[(dataset.user_id >= 6)&(dataset.user_id <= 9)]
#dataset=dataset.drop(columns=['video_id'])

    user_id like/dislike            video_id
0        17      dislike                [70]
1        20      dislike                [77]
2        20         like                [82]
3        22      dislike               [133]
4        22         like     [124, 129, 141]
5        24      dislike               [153]
6        27         like               [191]
7        33         like               [262]
8        41         like               [293]
9        42      dislike               [304]
10       49         like               [349]
11       51      dislike          [354, 369]
12       55         like               [387]
13       59         like               [415]
14       64         like               [450]
15       72      dislike               [524]
16       75      dislike               [544]
17       84         like               [651]
18       87         like          [675, 700]
19       90      dislike          [714, 719]
20       93      dislike               [736]
21       9

In [478]:
# split test and train set
from sklearn.model_selection import train_test_split

train_data, test_data = train_test_split(dataset, test_size=0.2)

In [479]:
train_data

,user_id,dislike,like,title_d,genre_d,watchTime,predict_labels
28,127,[1171],[1425],[1171],[7],[-0.11550981012523819],985
11,59,[1425],[415],[415],[0],[-0.12772265291003368],183
29,134,[1425],[1233],[1233],[8],[-0.12565300521181907],660
25,116,"[1021, 1035, 1055]",[1425],"[1021, 1035, 1055]","[8, 3]","[-0.0061989106136713145, 7.065509640315288, -0...",1252
8,49,[1425],[349],[349],[13],[-0.1456479764733658],980
1,20,[77],[82],"[77, 82]","[11, 10]","[-0.15838962043609195, -0.15311554845424225]",724
6,41,[1425],[293],[293],[6],[-0.15838962043609195],874
16,87,[1425],"[675, 700]","[675, 700]",[1],"[-0.11738635192698968, -0.15838962043609195]",633
10,55,[1425],[387],[387],[14],[-0.15838962043609195],775
33,141,[1425],[1377],[1377],[4],[-0.15681012345042528],1120


In [480]:
EMBEDDING_DIMS = 16
DENSE_UNITS = 64
DROPOUT_PCT = 0.0
ALPHA = 0.0
NUM_CLASSES=df["video_id"].max() + 3
LEARNING_RATE = 0.0003

In [481]:
#---inputs
import tensorflow as tf
import datetime
import os
input_title = tf.keras.Input(shape=(None, ), name='video_title')
inp_video_liked = tf.keras.layers.Input(shape=(None,), name='like')
inp_video_disliked = tf.keras.layers.Input(shape=(None,), name='dislike')
input_genre = tf.keras.Input(shape=(None, ), name='genre')
input_timestamp = tf.keras.Input(shape=(None, ), name='timestamp')
# input_occupation = tf.keras.Input(shape=(None, ), name='occupation')


#--- layers
features_embedding_layer = tf.keras.layers.Embedding(input_dim=NUM_CLASSES, output_dim=EMBEDDING_DIMS, 
                                            mask_zero=True, trainable=True, name='features_embeddings')
labels_embedding_layer = tf.keras.layers.Embedding(input_dim=NUM_CLASSES, output_dim=EMBEDDING_DIMS, 
                                            mask_zero=True, trainable=True, name='labels_embeddings')

avg_embeddings = MaskedEmbeddingsAggregatorLayer(agg_mode='mean', name='aggregate_embeddings')

dense_1 = tf.keras.layers.Dense(units=DENSE_UNITS, name='dense_1')
dense_2 = tf.keras.layers.Dense(units=DENSE_UNITS, name='dense_2')
dense_3 = tf.keras.layers.Dense(units=DENSE_UNITS, name='dense_3')
l2_norm_1 = L2NormLayer(name='l2_norm_1')

dense_output = tf.keras.layers.Dense(NUM_CLASSES, activation=tf.nn.softmax, name='dense_output')

#--- features
features_embeddings = features_embedding_layer(input_title)
l2_norm_features = l2_norm_1(features_embeddings)
avg_features = avg_embeddings(l2_norm_features)

labels_liked_embeddings = labels_embedding_layer(inp_video_liked)
l2_norm_liked = l2_norm_1(labels_liked_embeddings)
avg_liked = avg_embeddings(l2_norm_liked)

labels_disliked_embeddings = labels_embedding_layer(inp_video_disliked)
l2_norm_disliked = l2_norm_1(labels_disliked_embeddings)
avg_disliked = avg_embeddings(l2_norm_disliked)

labels_genre_embeddings = labels_embedding_layer(input_genre)
l2_norm_genre = l2_norm_1(labels_genre_embeddings)
avg_genre = avg_embeddings(l2_norm_genre)

labels_timestamp_embeddings = labels_embedding_layer(input_timestamp)
l2_norm_timestamp = l2_norm_1(labels_timestamp_embeddings)
avg_timestamp = avg_embeddings(l2_norm_timestamp)


# 임베딩 벡터들 연결
concat_inputs = tf.keras.layers.Concatenate(axis=1)([avg_features,
                                                     avg_liked,
                                                     avg_disliked,
                                                     avg_genre,
                                                     avg_timestamp
                                                     ])
# Dense Layers
dense_1_features = dense_1(concat_inputs)
dense_1_relu = tf.keras.layers.ReLU(name='dense_1_relu')(dense_1_features)
dense_1_batch_norm = tf.keras.layers.BatchNormalization(name='dense_1_batch_norm')(dense_1_relu)

dense_2_features = dense_2(dense_1_relu)
dense_2_relu = tf.keras.layers.ReLU(name='dense_2_relu')(dense_2_features)
# dense_2_batch_norm = tf.keras.layers.BatchNormalization(name='dense_2_batch_norm')(dense_2_relu)

dense_3_features = dense_3(dense_2_relu)
dense_3_relu = tf.keras.layers.ReLU(name='dense_3_relu')(dense_3_features)
dense_3_batch_norm = tf.keras.layers.BatchNormalization(name='dense_3_batch_norm')(dense_3_relu)
outputs = dense_output(dense_3_batch_norm)

#Optimizer
optimiser = tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE)

#--- prep model
model = tf.keras.models.Model(
    inputs=[input_title, 
            inp_video_liked, 
            inp_video_disliked,
            input_genre,
            input_timestamp,
            ],
    outputs=[outputs]
)
logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)
model.compile(optimizer=optimiser, loss='sparse_categorical_crossentropy', metrics=['acc'])

 #model.summary()

In [482]:
#train_data= train_data.fillna(0)
train_data

,user_id,dislike,like,title_d,genre_d,watchTime,predict_labels
28,127,[1171],[1425],[1171],[7],[-0.11550981012523819],985
11,59,[1425],[415],[415],[0],[-0.12772265291003368],183
29,134,[1425],[1233],[1233],[8],[-0.12565300521181907],660
25,116,"[1021, 1035, 1055]",[1425],"[1021, 1035, 1055]","[8, 3]","[-0.0061989106136713145, 7.065509640315288, -0...",1252
8,49,[1425],[349],[349],[13],[-0.1456479764733658],980
1,20,[77],[82],"[77, 82]","[11, 10]","[-0.15838962043609195, -0.15311554845424225]",724
6,41,[1425],[293],[293],[6],[-0.15838962043609195],874
16,87,[1425],"[675, 700]","[675, 700]",[1],"[-0.11738635192698968, -0.15838962043609195]",633
10,55,[1425],[387],[387],[14],[-0.15838962043609195],775
33,141,[1425],[1377],[1377],[4],[-0.15681012345042528],1120


In [483]:
history = model.fit([tf.keras.preprocessing.sequence.pad_sequences(train_data['title_d'])+ 1e-10,
           tf.keras.preprocessing.sequence.pad_sequences(train_data['like'])+ 1e-10,
           tf.keras.preprocessing.sequence.pad_sequences(train_data['dislike'])+ 1e-10,
            tf.keras.preprocessing.sequence.pad_sequences(train_data['genre_d'])+ 1e-10,
            tf.keras.preprocessing.sequence.pad_sequences(train_data['watchTime'], dtype=float) + 1e-5,
           ],train_data['predict_labels'].values,
           steps_per_epoch=1, epochs=500)

Epoch 1/500
1/1 [==============================] - 2s 2s/step - loss: 7.3040 - acc: 0.0000e+00
Epoch 2/500
1/1 [==============================] - 0s 4ms/step - loss: 7.2639 - acc: 0.0000e+00
Epoch 3/500
1/1 [==============================] - 0s 9ms/step - loss: 7.2269 - acc: 0.0000e+00
Epoch 4/500
1/1 [==============================] - 0s 5ms/step - loss: 7.1946 - acc: 0.0000e+00
Epoch 5/500
1/1 [==============================] - 0s 7ms/step - loss: 7.1640 - acc: 0.0000e+00
Epoch 6/500
1/1 [==============================] - 0s 5ms/step - loss: 7.1343 - acc: 0.0000e+00
Epoch 7/500
1/1 [==============================] - 0s 6ms/step - loss: 7.1057 - acc: 0.0000e+00
Epoch 8/500
1/1 [==============================] - 0s 7ms/step - loss: 7.0769 - acc: 0.0000e+00
Epoch 9/500
1/1 [==============================] - 0s 5ms/step - loss: 7.0486 - acc: 0.0000e+00
Epoch 10/500
1/1 [==============================] - 0s 6ms/step - loss: 7.0208 - acc: 0.0000e+00
Epoch 11/500
1/1 [======================

1/1 [==============================] - 0s 4ms/step - loss: 5.3058 - acc: 1.0000
Epoch 89/500
1/1 [==============================] - 0s 4ms/step - loss: 5.2840 - acc: 1.0000
Epoch 90/500
1/1 [==============================] - 0s 3ms/step - loss: 5.2623 - acc: 1.0000
Epoch 91/500
1/1 [==============================] - 0s 3ms/step - loss: 5.2404 - acc: 1.0000
Epoch 92/500
1/1 [==============================] - 0s 5ms/step - loss: 5.2184 - acc: 1.0000
Epoch 93/500
1/1 [==============================] - 0s 5ms/step - loss: 5.1963 - acc: 1.0000
Epoch 94/500
1/1 [==============================] - 0s 4ms/step - loss: 5.1742 - acc: 1.0000
Epoch 95/500
1/1 [==============================] - 0s 4ms/step - loss: 5.1518 - acc: 1.0000
Epoch 96/500
1/1 [==============================] - 0s 4ms/step - loss: 5.1296 - acc: 1.0000
Epoch 97/500
1/1 [==============================] - 0s 5ms/step - loss: 5.1072 - acc: 1.0000
Epoch 98/500
1/1 [==============================] - 0s 4ms/step - loss: 5.0848 - ac

1/1 [==============================] - 0s 4ms/step - loss: 3.2641 - acc: 1.0000
Epoch 176/500
1/1 [==============================] - 0s 5ms/step - loss: 3.2392 - acc: 1.0000
Epoch 177/500
1/1 [==============================] - 0s 4ms/step - loss: 3.2142 - acc: 1.0000
Epoch 178/500
1/1 [==============================] - 0s 4ms/step - loss: 3.1892 - acc: 1.0000
Epoch 179/500
1/1 [==============================] - 0s 3ms/step - loss: 3.1641 - acc: 1.0000
Epoch 180/500
1/1 [==============================] - 0s 4ms/step - loss: 3.1390 - acc: 1.0000
Epoch 181/500
1/1 [==============================] - 0s 3ms/step - loss: 3.1143 - acc: 1.0000
Epoch 182/500
1/1 [==============================] - 0s 5ms/step - loss: 3.0893 - acc: 1.0000
Epoch 183/500
1/1 [==============================] - 0s 4ms/step - loss: 3.0643 - acc: 1.0000
Epoch 184/500
1/1 [==============================] - 0s 3ms/step - loss: 3.0392 - acc: 1.0000
Epoch 185/500
1/1 [==============================] - 0s 4ms/step - loss: 3

1/1 [==============================] - 0s 3ms/step - loss: 1.2554 - acc: 1.0000
Epoch 263/500
1/1 [==============================] - 0s 4ms/step - loss: 1.2375 - acc: 1.0000
Epoch 264/500
1/1 [==============================] - 0s 4ms/step - loss: 1.2197 - acc: 1.0000
Epoch 265/500
1/1 [==============================] - 0s 4ms/step - loss: 1.2021 - acc: 1.0000
Epoch 266/500
1/1 [==============================] - 0s 4ms/step - loss: 1.1847 - acc: 1.0000
Epoch 267/500
1/1 [==============================] - 0s 4ms/step - loss: 1.1675 - acc: 1.0000
Epoch 268/500
1/1 [==============================] - 0s 4ms/step - loss: 1.1504 - acc: 1.0000
Epoch 269/500
1/1 [==============================] - 0s 5ms/step - loss: 1.1336 - acc: 1.0000
Epoch 270/500
1/1 [==============================] - 0s 4ms/step - loss: 1.1168 - acc: 1.0000
Epoch 271/500
1/1 [==============================] - 0s 4ms/step - loss: 1.1003 - acc: 1.0000
Epoch 272/500
1/1 [==============================] - 0s 3ms/step - loss: 1

1/1 [==============================] - 0s 4ms/step - loss: 0.3681 - acc: 1.0000
Epoch 350/500
1/1 [==============================] - 0s 4ms/step - loss: 0.3638 - acc: 1.0000
Epoch 351/500
1/1 [==============================] - 0s 4ms/step - loss: 0.3595 - acc: 1.0000
Epoch 352/500
1/1 [==============================] - 0s 4ms/step - loss: 0.3553 - acc: 1.0000
Epoch 353/500
1/1 [==============================] - 0s 4ms/step - loss: 0.3512 - acc: 1.0000
Epoch 354/500
1/1 [==============================] - 0s 3ms/step - loss: 0.3471 - acc: 1.0000
Epoch 355/500
1/1 [==============================] - 0s 5ms/step - loss: 0.3431 - acc: 1.0000
Epoch 356/500
1/1 [==============================] - 0s 4ms/step - loss: 0.3392 - acc: 1.0000
Epoch 357/500
1/1 [==============================] - 0s 4ms/step - loss: 0.3353 - acc: 1.0000
Epoch 358/500
1/1 [==============================] - 0s 5ms/step - loss: 0.3315 - acc: 1.0000
Epoch 359/500
1/1 [==============================] - 0s 4ms/step - loss: 0

1/1 [==============================] - 0s 3ms/step - loss: 0.1617 - acc: 1.0000
Epoch 437/500
1/1 [==============================] - 0s 4ms/step - loss: 0.1605 - acc: 1.0000
Epoch 438/500
1/1 [==============================] - 0s 4ms/step - loss: 0.1594 - acc: 1.0000
Epoch 439/500
1/1 [==============================] - 0s 3ms/step - loss: 0.1582 - acc: 1.0000
Epoch 440/500
1/1 [==============================] - 0s 4ms/step - loss: 0.1571 - acc: 1.0000
Epoch 441/500
1/1 [==============================] - 0s 3ms/step - loss: 0.1559 - acc: 1.0000
Epoch 442/500
1/1 [==============================] - 0s 4ms/step - loss: 0.1548 - acc: 1.0000
Epoch 443/500
1/1 [==============================] - 0s 3ms/step - loss: 0.1537 - acc: 1.0000
Epoch 444/500
1/1 [==============================] - 0s 4ms/step - loss: 0.1526 - acc: 1.0000
Epoch 445/500
1/1 [==============================] - 0s 5ms/step - loss: 0.1515 - acc: 1.0000
Epoch 446/500
1/1 [==============================] - 0s 3ms/step - loss: 0

In [484]:
results = model.evaluate([tf.keras.preprocessing.sequence.pad_sequences(test_data['title_d'])+ 1e-10,
           tf.keras.preprocessing.sequence.pad_sequences(test_data['like'])+ 1e-10,
           tf.keras.preprocessing.sequence.pad_sequences(test_data['dislike'])+ 1e-10,
            tf.keras.preprocessing.sequence.pad_sequences(test_data['genre_d'])+ 1e-10,
            tf.keras.preprocessing.sequence.pad_sequences(test_data['watchTime'], dtype=float) + 1e-10,
           ], test_data['predict_labels'].values+ 1e-10, verbose=1
        )

1/1 [==============================] - 1s 560ms/step - loss: 7.8771 - acc: 0.0000e+00


In [485]:
pred = model.predict([tf.keras.preprocessing.sequence.pad_sequences(test_data['title_d'])+ 1e-10,
           tf.keras.preprocessing.sequence.pad_sequences(test_data['like'])+ 1e-10,
           tf.keras.preprocessing.sequence.pad_sequences(test_data['dislike'])+ 1e-10,
            tf.keras.preprocessing.sequence.pad_sequences(test_data['genre_d'])+ 1e-10,
            tf.keras.preprocessing.sequence.pad_sequences(test_data['watchTime'], dtype=float) + 1e-10
           ])

In [486]:
pred

array([[0.00016766, 0.00015305, 0.00021068, ..., 0.00015404, 0.00018352,
        0.00015702],
       [0.00023308, 0.00025036, 0.00030166, ..., 0.00027886, 0.00030089,
        0.00027578],
       [0.00040345, 0.00035679, 0.00038449, ..., 0.00042612, 0.00033381,
        0.00042705],
       ...,
       [0.00053955, 0.00043493, 0.0005917 , ..., 0.00051978, 0.00043927,
        0.0004885 ],
       [0.00038814, 0.00039621, 0.00037392, ..., 0.00041822, 0.00033149,
        0.00050248],
       [0.00041025, 0.0005015 , 0.00051629, ..., 0.00050368, 0.00043836,
        0.00063871]], dtype=float32)

In [487]:
# ranking
###### 각 user당 top-3개의 추천 데이터를 뽑아낸다.
N = 3
k = np.sort((-pred).argsort()[:,:N])
k[k>new_data["video_id"].max()]=0
print(k)

[[  95  874 1351]
 [ 241  859  985]
 [ 207  651  868]
 [ 859  874  985]
 [ 190  498  985]
 [ 859  874  985]
 [ 190  207 1076]]
